In [42]:
from urllib.request import urlopen
import boto3

from __private import ACCESS_KEY
from __private import SECRET_KEY

import json
import os
# Please be careful that this line should be at last.
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')


In [43]:
with open("./data/pullFromTwitter/2017_01_04_00.json", "r") as f:
    firstLine = f.readline()
    firstLine_format = json.loads(firstLine)

In [44]:
firstLine_format

{'created_at': 'Wed Jan 04 05:00:10 +0000 2017',
 'id': 816509503109664768,
 'id_str': '816509503109664768',
 'text': 'Wind 20.9 km/h E. Barometer 1028.28 mb, Falling slowly. Temperature -0.9 °C. Rain today 0.0mm. Humidity 45%',
 'source': '<a href="http://sandaysoft.com/" rel="nofollow">Sandaysoft Cumulus</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 212967154,
  'id_str': '212967154',
  'name': 'Gordon Larsen',
  'screen_name': 'gordonlarsen',
  'location': 'Calgary, Alberta, Canada',
  'url': None,
  'description': None,
  'protected': False,
  'verified': False,
  'followers_count': 6,
  'friends_count': 37,
  'listed_count': 4,
  'favourites_count': 1,
  'statuses_count': 22090,
  'created_at': 'Sun Nov 07 15:45:02 +0000 2010',
  'utc_offset': -25200,
  'time_zone': 'Mountain Time (US & Canada)',
  'geo_enabled': True,
 

In [45]:
firstLine_format['place']

{'id': '30904a009a9000a0',
 'url': 'https://api.twitter.com/1.1/geo/id/30904a009a9000a0.json',
 'place_type': 'city',
 'name': 'Central Saanich',
 'full_name': 'Central Saanich, British Columbia',
 'country_code': 'CA',
 'country': 'Canada',
 'bounding_box': {'type': 'Polygon',
  'coordinates': [[[-123.484088, 48.550064],
    [-123.484088, 48.613903],
    [-123.359588, 48.613903],
    [-123.359588, 48.550064]]]},
 'attributes': {}}

In [46]:
firstLine_format['place']['name']

'Central Saanich'

In [47]:
def getPlaceName(jsonLine):
    return jsonLine['place']['name']

In [48]:
allTweetPlace = set()

In [49]:
fileName = "./data/pullFromTwitter/2017_01_04_00.json"
with open(fileName, "r") as f:
    everyLines = f.readlines()
    for line in everyLines:
        try:
            tweetInJson = json.loads(line)
            tweetPlace = getPlaceName(tweetInJson)
#             print(tweetPlace)
            allTweetPlace.add(tweetPlace)
        except:
            continue
#     everyLines_format = json.loads(everyLines)

In [50]:
print('length of allTweetPlace: ', len(allTweetPlace))

length of allTweetPlace:  945


In [51]:
print(allTweetPlace)

{'Rogers', 'South Windsor', 'Portsmouth', 'Albany', 'Union Park', 'Carnelian Bay', 'North Las Vegas', 'South Lebanon', 'Troy', 'Ottawa', 'Missouri', 'Arlington', 'Chesapeake', 'Brea', 'Highland', 'Tallahassee', 'Southborough', 'Estes Park', 'Cypress Gardens', 'Iowa', 'Baton Rouge', 'Bradenton', 'Fort Thomas', 'Hanford', 'Beverly Hills', 'Greensboro', 'Whitefish Bay', 'Des Moines', 'Bowen Island', 'Riverside', 'Branson', 'Santa Catarina', 'Hoboken', 'Piedras Negras', 'The Blue Mountains', 'Clermont', 'Dover', 'Gering', 'Edmond', 'Upland', 'Marinette', 'Tinton Falls', 'Kinston', 'Holiday City South', 'The Las Vegas Strip', 'Parma', 'Lancaster', 'Carmichael', 'New Mexico', 'New Brunswick', 'North Portal', 'Mansfield', 'Gadsden', 'Trenton', 'Howell', 'Lynwood', 'North Bergen', 'Lake City', 'Cardiff', 'New Britain', 'Martinsville', 'Evansville', 'East Lake', 'Kerrville', 'Ocala', 'Lansing', 'Deltona', 'Highland Park', 'Enterprise', 'Guadalupe', 'Burlington', 'Town and Country', 'Pocatello',

We can see there are also nearby city in the tweet's location. For example, Mill Valley, Sausalito and San Rafael appears here. Following I will try to import all pull data from server.

Here we only take a month period data. January 2017. 

In [52]:
def checkSFTweets(fileName):
    SF_tweetsInOneFile = []
    with open(fileName, "r") as f:
        everyLines = f.readlines()
        for line in everyLines:
            try:
                tweetInJson = json.loads(line)
                tweetPlace = getPlaceName(tweetInJson)
                if tweetPlace == 'San Francisco':
                    SF_tweetsInOneFile.append(line)
            except:
                continue
        return SF_tweetsInOneFile

In [60]:
%%time
# Iterate the files in the directory you stored the data.
jsonFileDirectoryPath = "C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol/data/pullFromTwitter_2017_01"
jsonFiles= os.listdir(jsonFileDirectoryPath)
# Create a list to store SF tweets
SF_tweets = []

# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol/data/pullFromTwitter_2017_01')
for jsonFile in jsonFiles:
    # Make sure we open the file that is Json format (not .tar) and not directory.
    if not os.path.isdir(jsonFile) and os.path.splitext(jsonFile)[-1] == ".json":
        # Iterate the jsonFile to check if there are any SF tweets
        SF_tweetsInOneFile = checkSFTweets(jsonFile)
        SF_tweets.extend(SF_tweetsInOneFile)


C:\Users\eddie\Desktop\Prof. Rumi Chunara\alcohol
Wall time: 8min 45s


Remove duplicate, even it's only a small part. 

In [72]:
SF_tweets = list(set(SF_tweets))

In [73]:
print('Tweets that posted in San Francisco: ', len(SF_tweets))
# print(SF_tweets[1])

Tweets that posted in San Francisco:  87764


In [74]:
# Write SF tweets into file.
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
SF_tweetsFileName = './data/SF_tweets_2017_01.json'
with open(SF_tweetsFileName, "w") as f:
    for tweet in SF_tweets:
        f.write(tweet)

In [75]:
# Load those tweets back in Json format to check if there are anything wrong.
with open("./data/SF_tweets_2017_01.json", "r") as f:
    firstLine = f.readline()
    firstLine_format = json.loads(firstLine)
#     everyLines = f.readlines()
#     for line in everyLines:
#         tweetInJson = json.loads(line)
    

In [76]:
firstLine_format

{'created_at': 'Wed Jan 04 21:51:34 +0000 2017',
 'id': 816764032929660928,
 'id_str': '816764032929660928',
 'text': 'Opened Street or Sidewalk Cleaning request via android at 3154 17th St https://t.co/nR9l9sWgkf. Piles of trash. https://t.co/rYeD6jSgnr',
 'display_text_range': [0, 111],
 'source': '<a href="http://spotreporters.com" rel="nofollow">Spot Reporters Server</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1589692776,
  'id_str': '1589692776',
  'name': 'SF311 Reports',
  'screen_name': 'SF311Reports',
  'location': 'San Francisco, CA',
  'url': 'http://mobile311.sfgov.org',
  'description': "This is account is not monitored and is used to post service request update information. To communicate with SF311, please use our main account 'SF311'",
  'protected': False,
  'verified': False,
  'followers_count': 128,
  'f

There are 87764 tweets posted in San Francisco in January 2017.